### Install Needed Libraries


In [3]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00:00:0100:01
  Attempting un

### Load the Medical Chatbot Dataset


In [4]:
import pandas as pd
df = pd.read_csv("/kaggle/input/ai-medical-chatbot/ai-medical-chatbot.csv")

### More Information About Df

In [5]:
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256916 entries, 0 to 256915
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Description  256916 non-null  object
 1   Patient      256916 non-null  object
 2   Doctor       256916 non-null  object
dtypes: object(3)
memory usage: 5.9+ MB


In [7]:
df.describe()

,Description,Patient,Doctor
count,256916,256916,256916
unique,228722,246006,242150
top,Q. For how long should I take Kalachikai powde...,"Hello doctor,I am trying to conceive but my hu...",Hi. For further doubts consult a sexologist on...
freq,1137,1137,1519


In [8]:
df.isnull().sum()

Description    0
Patient        0
Doctor         0
dtype: int64

In [9]:
df.duplicated().sum()

10378

In [10]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [11]:
## take consideration only first 10000 Row. 
# df = df[:10000]

### Prepare Conversation Format (Patient ↔ Doctor)

In [12]:
def prepared_conversation_between_doctor_and_patient(row):
    patient_question = "## Patient: " + row["Description"].strip()
    doctor_answer = "## Doctor: " + row["Doctor"].strip()
    conversation = {
        "patient_question" : patient_question,
        "doctor_answer" : doctor_answer
    }
    return conversation

In [13]:
conversations = df.apply(prepared_conversation_between_doctor_and_patient, axis=1)
print("Number of Conversations between Doctor and Patients: ", len(conversations))

Number of Conversations between Doctor and Patients:  246538


In [14]:
conversations.tolist()[:20]

[{'patient_question': '## Patient: Q. What does abutment of the nerve root mean?',
  'doctor_answer': '## Doctor: Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'},
 {'patient_question': '## Patient: Q. What should I do to reduce my weight gained due to genetic hypothyroidism?',
  'doctor_answer': '## Doctor: Hi. You have really done well with the hypothyroidism problem. Your levels are normal with less medications which are very good. As it is genetically induced, it is very difficult to lose weight. My advice to you is, you should focus on maintaining normal levels of TSH (thyroid-stimulating hormone) and try to remain active, having a positive outlook in life. Or else, it will become very difficult to balance your life with the symptoms of hypothyroidism. Even though your weight has not reduced, be very careful in not putting on weight here afterward. Everyday brisk walk

### Convert Conversations into Hugging Face Dataset

In [15]:
# convert prepared input (patient questions & doctor ansers ) into datadfame.
from datasets import Dataset

dataset = Dataset.from_list(conversations.tolist())

print("Number of training examples:", len(dataset))
print(dataset[0])

Number of training examples: 246538
{'patient_question': '## Patient: Q. What does abutment of the nerve root mean?', 'doctor_answer': '## Doctor: Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}


### Import Transformers & Suppress Warnings

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import transformers
import warnings
warnings.filterwarnings("ignore")

### Split Dataset into Training (80%) and Validation (20%)

In [17]:
# Training dataset (80%)
fraction = 0.8
train_size = int(fraction * len(dataset))

train_dataset = dataset.shuffle(seed=42).select(range(train_size))

# Validation dataset (20%)
eval_dataset = dataset.shuffle(seed=42).select(range(train_size, len(dataset)))

In [18]:
print(f"Train size: {len(train_dataset)}")
print(f"Eval size: {len(eval_dataset)}")

Train size: 197230
Eval size: 49308


### Load Base Model and Tokenizer + Define Tokenization Function


In [19]:
base_model_id = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModelForCausalLM.from_pretrained(base_model_id)



tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(conversations):
    texts = [
        patient_question + "\n" + doctor_answer
        for patient_question, doctor_answer in zip(conversations['patient_question'], conversations['doctor_answer'])
    ]
    tokenized_texts = tokenizer(texts, truncation=True, padding="max_length", max_length= 512 )
    tokenized_texts["labels"] = tokenized_texts["input_ids"].copy()
    return tokenized_texts

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [20]:
# Tokenize the dataset
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")

Map:   0%|          | 0/197230 [00:00<?, ? examples/s]

Map:   0%|          | 0/49308 [00:00<?, ? examples/s]

### Define Training Arguments and Initialize Trainer


In [50]:
output_dir = "./fine_tuned_medical_chatbot"

training_args = TrainingArguments(
    output_dir=output_dir,
    max_steps=300,                 
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=1,                
    save_strategy="steps",           
    save_steps=100,
    save_total_limit=2,              
    disable_tqdm=False,
    report_to="none",             
)

trainer = Trainer(
    model = model,
    args = training_args, 
    train_dataset = tokenized_train,
    # eval_dataset = tokenized_eval

)


PyTorch: setting up devices
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [51]:
## enable checkpointing + CUDA memory fix
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
model.gradient_checkpointing_enable()

### Start Fine-Tuning the Model


In [52]:
print("Starting training...")
trainer.train()
print("Training complete.")

The following columns in the Training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: doctor_answer, patient_question. If doctor_answer, patient_question are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 197,230
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Training with DataParallel so batch size has been adjusted to: 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 300
  Number of trainable parameters = 354,823,168


Starting training...


Step,Training Loss
1,0.008800
2,0.005000
3,0.005000
4,0.029100
5,0.006100
6,0.005700
7,0.036800
8,0.073200
9,0.223800
10,0.051300


Saving model checkpoint to ./fine_tuned_medical_chatbot/checkpoint-100
Configuration saved in ./fine_tuned_medical_chatbot/checkpoint-100/config.json
Configuration saved in ./fine_tuned_medical_chatbot/checkpoint-100/generation_config.json
Model weights saved in ./fine_tuned_medical_chatbot/checkpoint-100/model.safetensors
Saving model checkpoint to ./fine_tuned_medical_chatbot/checkpoint-200
Configuration saved in ./fine_tuned_medical_chatbot/checkpoint-200/config.json
Configuration saved in ./fine_tuned_medical_chatbot/checkpoint-200/generation_config.json
Model weights saved in ./fine_tuned_medical_chatbot/checkpoint-200/model.safetensors
Saving model checkpoint to ./fine_tuned_medical_chatbot/checkpoint-300
Configuration saved in ./fine_tuned_medical_chatbot/checkpoint-300/config.json
Configuration saved in ./fine_tuned_medical_chatbot/checkpoint-300/generation_config.json
Model weights saved in ./fine_tuned_medical_chatbot/checkpoint-300/model.safetensors
Deleting older checkpoint

Training complete.


### Save Fine-Tuned Model and Tokenizer


In [53]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model and tokenizer saved Successfully.")

Saving model checkpoint to ./fine_tuned_medical_chatbot
Configuration saved in ./fine_tuned_medical_chatbot/config.json
Configuration saved in ./fine_tuned_medical_chatbot/generation_config.json
Model weights saved in ./fine_tuned_medical_chatbot/model.safetensors
chat template saved in ./fine_tuned_medical_chatbot/chat_template.jinja
tokenizer config file saved in ./fine_tuned_medical_chatbot/tokenizer_config.json
Special tokens file saved in ./fine_tuned_medical_chatbot/special_tokens_map.json


Model and tokenizer saved Successfully.


### Reload Fine-Tuned Medical Chatbot Model for Inference

In [58]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_dir = "/kaggle/working/fine_tuned_medical_chatbot"

# Load model & tokenizer
model = AutoModelForCausalLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Create a chatbot pipeline
chatbot = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0
)

loading configuration file ./fine_tuned_medical_chatbot/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights file ./fine_tuned_medical_chatbot/

### Test the Fine-Tuned Medical Chatbot


In [61]:
prompt = "Patient: What are the symptoms of Lymphocytic Choriomeningitis (LCM)?\nDoctor:"
response = chatbot(prompt, max_length=200, do_sample=True, top_p=0.9, temperature=0.8)

print("Chatbot:", response[0]["generated_text"])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chatbot: Patient: What are the symptoms of Lymphocytic Choriomeningitis (LCM)?
Doctor: Hello.LCM is a skin infection with high risk of viral infection.It can be present in both wound and soft tissue tissue of the body.It is characterized by high rate of redness and swelling.I would suggest you to consult a dermatologist or an ENT physician for better treatment.


In [106]:
# 🌟 Interactive loop to test the chatbot 🌟
print("\n🤖 Medical Chatbot is ready to assist you.")
print("💬 Type 'exit' or 'quit' or 'stop' anytime to stop.\n")

while True:
    user_input = input("👤 You: ").strip()
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("\n👋 Session ended. Stay healthy!")
        break
        
    # Format the input to indicate patient dialogue, and prompt the model for a doctor's reply.
    formatted_input = f"Patient: {user_input}\nDoctor:"
    response = chatbot(
        formatted_input,
        max_length=150,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Extract doctor's reply by taking the text after "Doctor:" if present.
    generated_text = response[0]['generated_text']
    if "Doctor:" in generated_text:
        doctor_reply = generated_text.split("Doctor:")[-1].strip()
    else:
        doctor_reply = generated_text.strip()
    
    # Print doctor's reply with nice formatting
    print("🩺 Doctor:", doctor_reply)
    print("-" * 60)  # Divider for readability


🤖 Medical Chatbot is ready to assist you.
💬 Type 'exit' or 'quit' or 'stop' anytime to stop.



👤 You:  What are the symptoms of diabetes?


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🩺 Doctor: Hi,Welcome to HCM.Diabetes is a condition that causes blood sugar to drop.Diabetes can cause blood sugar to drop.The symptoms of diabetes is high cholesterol levels,high blood pressure, high blood pressure, diabetes.Diabetes can also cause a low blood cholesterol level.It is a common cause of diabetes.You are diagnosed with diabetes.You can avoid it by eating healthy and getting regular exercise.You can also avoid it by avoiding sugar.I hope this information helps you in choosing a doctor for diabetes.I will be happy to help you further.
------------------------------------------------------------


👤 You:  How is hypertension diagnosed?


...

👤 You:  What are the common symptoms of COVID-19?


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🩺 Doctor: Hi,I am Dr.Rajesh V.Thanks for your query.I have gone through your query.I can understand your concerns.The common symptoms of COPD are headache and eye pain.The common symptoms of COVID are eye pain, headache, and eye fatigue.The common symptoms of COPD are eye pain, headache, and eye fatigue.The common symptoms of COPD are headache, eye pain, and eye fatigue.Thanks for the query.I will be happy to answer your questions.Take care.Dr.Rajeshit V.
------------------------------------------------------------


👤 You:  exit



👋 Session ended. Stay healthy!
